In [ ]:
import torch
import pickle

import circuits.analysis as analysis
import circuits.eval_sae_as_classifier as eval_sae
import circuits.chess_utils as chess_utils

In [ ]:
import torch

# Define a sample 3D tensor with random values
# Dimensions: T x F x C (let's use 2 x 3 x 4 for simplicity)
f1_TFC = torch.randn(2, 3, 4)
print("Original Tensor (T x F x C):")
print(f1_TFC)

def best_f1_average(f1_TFC: torch.Tensor) -> torch.Tensor:
    # Apply torch.max along the last dimension (dimension 2)
    # Select only the values, ignoring the indices
    f1_TF, _ = torch.max(f1_TFC, dim=1)
    return f1_TF

# Compute the maximum along the 'C' dimension and reduce to a 2D tensor
f1_TF = best_f1_average(f1_TFC)
print("\nReduced Tensor (T x F) with max values from 'C':")
print(f1_TF)

In [ ]:
def mask_all_blanks(results: dict, device) -> dict:
    custom_functions = analysis.get_all_custom_functions(results)
    for function in custom_functions:
        function_name = function.__name__

        if function == chess_utils.board_to_piece_state or function == chess_utils.board_to_piece_color_state:
            on_TFRRC = results[function_name]['on']
            off_TFRRC = results[function_name]['off']
            results[function_name]['on'] = analysis.mask_initial_board_state(on_TFRRC, function, device)
            results[function_name]['off'] = analysis.mask_initial_board_state(off_TFRRC, function, device)

    return results

def best_f1_average(f1_TFRRC: torch.Tensor) -> torch.Tensor:
    f1_TRRC, _ = torch.max(f1_TFRRC, dim=1)
    


In [ ]:

device = "cpu"
autoencoder_group_paths = ["../autoencoders/chess_layer5/"]
folders = eval_sae.get_nested_folders(autoencoder_group_paths[0])

custom_functions = []

sae_results = {}

for folder in folders:

    sae_results[folder] = {}

    results_filenames = analysis.get_all_results_file_names(folder)
    if len(results_filenames) > 1 or len(results_filenames) == 0:
        print(f"Skipping {folder} because it has {len(results_filenames)} results files")
        print("This is most likely because there are results files from different n_inputs")
        continue
    results_filename = results_filenames[0]

    evals_filename = analysis.get_all_evals_file_names(folder)

    if len(evals_filename) > 1 or len(evals_filename) == 0:
        print(f"Skipping {folder} because it has {len(evals_filename)} evals files")
        print("This is most likely because there are evals files from different n_inputs")
        continue

    evals_filename = evals_filename[0]

    with open(results_filename, "rb") as f:
        results = pickle.load(f)

    with open(evals_filename, "rb") as f:
        evals = pickle.load(f)
    
    results = mask_all_blanks(results, device)
    f1_dict_TFRRC = analysis.get_all_f1s(results, device)

    if len(custom_functions) == 0:
        for key in f1_dict_TFRRC:
            custom_functions.append(key)
    else:
        for key in f1_dict_TFRRC:
            assert key in custom_functions, f"Key {key} not in custom_functions"

    sae_results[folder]["l0"] = evals["l0"]
    sae_results[folder]["frac_variance_explained"] = evals["frac_variance_explained"]

    for custom_function in f1_dict_TFRRC:
        f1_TFRRC = f1_dict_TFRRC[custom_function]

